In [94]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, Activation, Reshape, MaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers.convolutional import Conv1D, Conv2D
from keras import optimizers
import sys
import keras.utils
import os
sys.path.append(os.path.join(sys.path[0], '../utils'))
from load_mat_file import load_mat_file
from keras.callbacks import TensorBoard
import copy
import matplotlib.pyplot as plt
%matplotlib inline

In [70]:
def create_network(input_dim, num_classes):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,)))
    seq.add(Activation('relu'));
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(num_classes, activation='relu'))
    model.add(Activation('softmax'))
    return seq


def create_network_tu(max_words, num_classes):
    model = Sequential()
    model.add(Dense(512, input_shape=(max_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model;


def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()


def create_network_eeg_conv1(input_dim, num_classes):
    seq = Sequential()
    seq.add(Reshape(( input_dim, 1,), input_shape=(input_dim,)))
    seq.add(Conv1D(filters=16, kernel_size=5, activation='relu', strides=2))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 5, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 3, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 3, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(GlobalAveragePooling1D())
    #seq.add(Reshape(11328,1))
    seq.add(Dense(units=128))
    seq.add(Activation('relu'));
    seq.add(Dense(units=num_classes))
    #seq.add(Reshape((num_classes,)))
    seq.add(Activation('softmax'));
    return seq;

def create_network_eeg_conv1_multichannel(input_dim, num_channels, num_classes):
    seq = Sequential()
    #seq.add(Reshape(( input_dim, 1,), input_shape=(input_dim,)))
    seq.add(Conv1D(filters=64, kernel_size=5, activation='relu', strides=2, input_shape=(input_dim,num_channels,)))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 5, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 3, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 3, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(GlobalAveragePooling1D())
    #seq.add(Reshape(11328,1))
    seq.add(Dense(units=128))
    seq.add(Activation('relu'));
    seq.add(Dense(units=num_classes))
    #seq.add(Reshape((num_classes,)))
    seq.add(Activation('softmax'));
    return seq;

In [ ]:
def create_cnn_mc_small(input_dim, num_channels, num_classes):
    seq = Sequential()
    #seq.add(Reshape(( input_dim, 1,), input_shape=(input_dim,)))
    seq.add(Conv1D(filters=64, kernel_size=5, activation='relu', strides=2, input_shape=(input_dim,num_channels,)))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 5, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    #seq.add(Conv1D(32, 3, activation='relu'))
    #seq.add(MaxPooling1D(3, strides=2))
    #seq.add(Conv1D(32, 3, activation='relu'))
    #seq.add(MaxPooling1D(3, strides=2))
    print (seq.output_shape)
    seq.add(Reshape((seq.output_shape[1]*seq.output_shape[2]) ))
    seq.add(GlobalAveragePooling1D())
    #seq.add(Reshape(11328,1))
    seq.add(Dense(units=128))
    seq.add(Activation('relu'));
    seq.add(Dense(units=num_classes))
    #seq.add(Reshape((num_classes,)))
    seq.add(Activation('softmax'));
    return seq;

model = create_cnn_mc_small(size_of_sample, number_of_channels, num_classes)
#model = create_network_eeg_conv1_multichannel(size_of_sample, number_of_channels, num_classes)
print(model.summary())


In [105]:
def load_data():
    EXP_NUM = 21 # total number of people
    EXP_SHIFT = 167 # shift in numerations
    CHANNELS_NUM = 21 # number of channels in EEG record
    sel_channels = slice(14, 18)
    all_data = np.array([])
    all_Y = np.array([])
    for eeg_num in range(1, EXP_NUM + 1):
        EEG_FILENAME = 'eegmat_selected/D0000' + str(eeg_num + EXP_SHIFT)
        EEG = load_mat_file(EEG_FILENAME, 's')
        X_data = EEG["eeg"][0][0]
        X_data = np.swapaxes(X_data, 2, 0)
        X_data = np.swapaxes(X_data, 1, 2)
        Y_data = EEG["mrk"][0][0]
        OneChannel_data = X_data[:, sel_channels, :]
        OneChannel_data = OneChannel_data.reshape(OneChannel_data.shape[0], OneChannel_data.shape[1]*OneChannel_data.shape[2])
        if eeg_num == 1:
            all_data = np.array([]).reshape(0, OneChannel_data.shape[1])
            all_Y = np.array([]).reshape(0, Y_data.shape[1])
        all_data = np.concatenate([all_data, OneChannel_data])
        all_Y = np.concatenate([all_Y, Y_data])

In [74]:
def load_all_data(data_path):
    EXP_NUM = 21 # total number of people
    EXP_SHIFT = 167 # shift in numerations
    CHANNELS_NUM = 21 # number of channels in EEG record
    sel_channels = slice(14, 18)
    all_data = np.array([])
    all_Y = np.array([])
    for eeg_num in range(1, EXP_NUM + 1):
        EEG_FILENAME = 'eegmat_selected/D0000' + str(eeg_num + EXP_SHIFT)
        EEG = load_mat_file(os.path.join(data_path,EEG_FILENAME), 's')
        X_data = EEG["eeg"][0][0]
        
        X_data = np.swapaxes(X_data, 2, 0)
        X_data = np.swapaxes(X_data, 1, 2)
        Y_data = EEG["mrk"][0][0]
        OneChannel_data = X_data[:, sel_channels, :]
        OneChannel_data = OneChannel_data.reshape(OneChannel_data.shape[0], OneChannel_data.shape[1]*OneChannel_data.shape[2])
        if eeg_num == 1:
            all_data = np.array([]).reshape(0, OneChannel_data.shape[1])
            all_Y = np.array([]).reshape(0, Y_data.shape[1])
        all_data = np.concatenate([all_data, OneChannel_data])
        all_Y = np.concatenate([all_Y, Y_data])
    return all_data, all_Y;

In [182]:
def show_data(X_data, Y_data, indx, channels):
    sample = X_data[indx];
    label = Y_data[indx];
    f, ax = plt.subplots(figsize=(8, 8))
    x = range(0, sample.shape[0]);
    for i in channels:
        plt.plot(x, sample[:,i])
        
    ax.legend(['ch ' + repr(i) for i in channels])
    ax.set_title('stimulus numer: ' + repr(indx) + ', label: ' + repr(Y_data[indx]));
    ax.set_xlabel('time, ticks')
    ax.set_ylabel('voltage')
        #x = range(0, sample.shape[0])
        #plt.xlabel('milliseconds');
        #plt.ylabel('volatage');
        
        #ax.plot(acc_levels, acc_perc, color='steelblue', label='perceptron')
        #ax.plot(acc_levels, acc_lstm, color='orangered', label='lstm, exp avg')
        #ax.plot(acc_levels, acc_lstm_2, color='orangered', label='lstm, 0 pred', linestyle='--')
        # ax.plot(acc_levels, acc_merged, color='seagreen')


        #ax.set_xlabel("Distance between predicted and actual dot")
        #ax.set_ylabel("Amount of predictions within the distance")
        # plt.vlines([0.1, 0.2, 0.3], 0, 1, label="within 0.1 of screen size", colors='gray', linestyles='--')
        # plt.hlines([0.25, 0.5, 0.75], 0, 1, label="within 0.1 of screen size", colors='gray', linestyles='--')

        #ax.grid(which='both')                                                            

        # or if you want differnet settings for the grids:                               
        #ax.grid(which='minor', alpha=0.1)                                                
        #ax.grid(which='major', alpha=0.5)   
        #ax.minorticks_on()
        #plt.legend()

#show_data(X_data, Y_data, 2, [0,18])

In [142]:
def load_all_data_multichannel(data_path):
    EXP_NUM = 21 # total number of people
    EXP_SHIFT = 167 # shift in numerations
    CHANNELS_NUM = 21 # number of channels in EEG record
    sel_channels = slice(0, 18)
    sel_time = slice(0,60,1) #quanta time
    all_data = np.array([])
    all_Y = np.array([])
    for eeg_num in range(1, EXP_NUM + 1):
        EEG_FILENAME = 'eegmat_selected/D0000' + str(eeg_num + EXP_SHIFT)
        EEG = load_mat_file(os.path.join(data_path,EEG_FILENAME), 's')
        X_data = EEG["eeg"][0][0]
        #print(X_data[:,:,1])
        #print(X_data.shape)
        X_data = np.swapaxes(X_data, 2, 0)
        X_data = np.swapaxes(X_data, 1, 2)
        
        Y_data = EEG["mrk"][0][0]
        #show_data(X_data, Y_data, 1, [0,2])
        OneChannel_data = X_data[:, sel_time, sel_channels]
        #OneChannel_data = OneChannel_data.reshape(OneChannel_data.shape[0], OneChannel_data.shape[1]*OneChannel_data.shape[2])
        if eeg_num == 1:
            all_data = np.array([]).reshape(0, OneChannel_data.shape[1], OneChannel_data.shape[2])
            all_Y = np.array([]).reshape(0, Y_data.shape[1])
        all_data = np.concatenate([all_data, OneChannel_data])
        all_Y = np.concatenate([all_Y, Y_data])
    return all_data, all_Y;

In [140]:
data_path = '/workspace/data/eeg/'

In [143]:
X_data, Y_data = load_all_data_multichannel(data_path);
number_of_samples = X_data.shape[0];
size_of_sample = X_data.shape[1];
number_of_channels = X_data.shape[2];

print('number of samples:', number_of_samples)
print('time counts:', size_of_sample)
print('number_of_channels:', number_of_channels)

Y_data = Y_data - 1;
Y_data = Y_data.reshape(Y_data.shape[0])
num_classes = int(max(Y_data) + 1);
print('num_classes: ', num_classes)
y_= keras.utils.to_categorical(Y_data, int(num_classes));
show_data(X_data, Y_data, [1,15])

number of samples: 3703
time counts: 60
number_of_channels: 4
num_classes:  2


In [144]:
x_= X_data;
test_prop = 0.1;
test_size= round(test_prop * number_of_samples, 0);
test_size = int(test_size)
train_size = number_of_samples - test_size;

In [145]:
#random shuffle of dataset and train-test split
idx = np.random.choice(np.arange(x_.shape[0]), x_.shape[0], replace=False)
idx_train = idx[0:train_size]
idx_test = idx[train_size:]
print('train size: ', train_size)
print('test size: ', test_size)
y_train = y_[idx_train];
x_train = x_[idx_train];
y_test= y_[idx_test];
x_test= x_[idx_test];

train size:  3333
test size:  370


In [87]:
x_train.shape

(3333, 275, 21)

In [156]:
model.output_shape

(None, 2)

In [152]:
model = create_cnn_mc_small(size_of_sample, number_of_channels, num_classes)
#model = create_network_eeg_conv1_multichannel(size_of_sample, number_of_channels, num_classes)
print(model.summary())
#model = create_network_tu(size_of_sample, num_classes)
#print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_44 (Conv1D)           (None, 28, 64)            1344      
_________________________________________________________________
max_pooling1d_42 (MaxPooling (None, 13, 64)            0         
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 9, 32)             10272     
_________________________________________________________________
max_pooling1d_43 (MaxPooling (None, 4, 32)             0         
_________________________________________________________________
global_average_pooling1d_11  (None, 32)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               4224      
_________________________________________________________________
activation_24 (Activation)   (None, 128)               0         
__________

In [89]:
batch_size = 20;
epochs = 5;
sgd = optimizers.Adagrad(lr=0.00001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[tensorboard])

score = model.evaluate(x_test, y_test, batch_size=batch_size)
print(score)

Train on 2666 samples, validate on 667 samples
Epoch 1/5
2666/2666 [==============================] - 2s 601us/step - loss: 0.6995 - acc: 0.5023 - val_loss: 0.6996 - val_acc: 0.4903
Epoch 2/5
2666/2666 [==============================] - ETA: 0s - loss: 0.6980 - acc: 0.501 - 2s 565us/step - loss: 0.6981 - acc: 0.5011 - val_loss: 0.6994 - val_acc: 0.4798
Epoch 3/5
2666/2666 [==============================] - 2s 564us/step - loss: 0.6975 - acc: 0.5041 - val_loss: 0.6992 - val_acc: 0.4843
Epoch 4/5
2666/2666 [==============================] - 1s 563us/step - loss: 0.6971 - acc: 0.5030 - val_loss: 0.6991 - val_acc: 0.4843
Epoch 5/5
370/370 [==============================] - 0s 198us/step
[0.7115293821773013, 0.4432432417934005]
